## Generate Data

In [ ]:
#generate fake data
import numpy as np
import pandas as pd
import random
import sys, os
import math
import urllib.request

import folium # last version 0.11.0
from folium import plugins

from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

import geopandas as gpd

In [ ]:
# Load Morocco geojson (replace with the correct one)
# We will generate points insde list of regions 

morocco_map = gpd.read_file('./datasets/maroc.geojson')
regions_map = gpd.read_file('./datasets/regions.geojson')
communes_map = gpd.read_file('./datasets/communes.geojson')
irfane_map = gpd.read_file('./datasets/irfane.geojson')

morocco_polygon = morocco_map["geometry"][0]
regions_polygons = regions_map["geometry"]

irfane_polygon = irfane_map["geometry"]
regions_polygons[0].centroid.y

In [ ]:
# generate a dictionaty data structure of user's latitude, longitude and score over time inside a list of regions
# input the initial position, the number of users, radius (chose a big radius to cover big regions), 
# time (integer), and list of the polygons of the regions
def generate_random_data(init_longitude = 0.0, init_latitude = 0.0, radius = 5, num_users = 1, time = 1, list_polygons = None):
    data = {}
    #cumuleScoreDict = {}
    for ts in np.arange(time):
        new_ts = {}
        for uid in np.arange(num_users):

            u = random.random()
            v = random.random()
            w = radius * np.sqrt(u)
            t = 2 * np.pi * v
            epsilon_longitude = w * np.sin(t)
            epsilon_latitude = w * np.cos(t) 
            
            # costruct new point
            new_longitude = init_longitude + epsilon_longitude
            new_latitude = init_latitude + epsilon_latitude
            
            #check if the constructed point is inside region_polygon
            new_point = Point(new_latitude, new_longitude)

            randomScore = random.random()
            
            idx_poly = contains(list_polygons, new_point)
            
            if idx_poly != None:
                new_longitude = init_longitude + epsilon_longitude
                new_latitude = init_latitude + epsilon_latitude
            else: # just keep initial position
                new_longitude = init_longitude
                new_latitude = init_latitude
            
            # create user data
            new_uid = {
                #"uid":uid,
                "longitude": new_longitude, 
                "latitude": new_latitude, 
                "score":randomScore,
                "idx_poly": idx_poly
                }
            new_ts["uid" + str(uid)] = new_uid
                
        # append timestamp
        data[str(ts)] = new_ts
        
    return data

In [ ]:
# test if of the gerated point is inside a list of polygons
def contains(list_polygons, point):
    for idx_poly, polygon in enumerate(list_polygons):
        if polygon.contains(point):
            return idx_poly
    return None

In [ ]:
#region of interest
#Test 0: Chose all morocco
LIST_POLYGONS  = [morocco_polygon]
#Test 1: Chose from the twelve regions
#  Tanger (0),Oujda (1),Rabat (2), (3), Casa (4), BeniMellal(5), Marakech(6), Errachidia(7), ...
LIST_POLYGONS  = [regions_polygons[4], regions_polygons[2]]
# Test 2: chose irfane region
LIST_POLYGONS  = irfane_polygon

# initial Points  (Rabat)
INIT_LONGITUDE = LIST_POLYGONS[1].centroid.y
INIT_LATITUDE  = LIST_POLYGONS[1].centroid.x

# simulation parameters
NUM_USERS = 1000
TIME = 50 # in seconds

In [ ]:
# Generate data
data = generate_random_data( 
    init_longitude = INIT_LONGITUDE, 
    init_latitude = INIT_LATITUDE,
    radius = .05, # radius tested: 0.05 for Irfane, 5 for regions, 15 for Morocco
    num_users = NUM_USERS, 
    time = TIME,
    list_polygons = LIST_POLYGONS
)

## Use Cholorpleth With Folium

In [ ]:
# We need to prapare data structure for this task
data_polygons = [[[0 for k in np.zeros(3)] for j in np.zeros(len(LIST_POLYGONS))] for i in np.zeros(TIME)]

for ts in np.arange(TIME):
    for key, value in data[str(ts)].items():
        idx_poly = value['idx_poly']
        if idx_poly != None:
            data_polygons[ts][idx_poly][0] = LIST_POLYGONS[idx_poly].centroid.y #longitude
            data_polygons[ts][idx_poly][1] = LIST_POLYGONS[idx_poly].centroid.x #latitude
            data_polygons[ts][idx_poly][2] += value['score'] # cumulative score

In [ ]:
med_lon_roi, med_lat_roi, med_score_roi = np.median(np.median(np.array(data_polygons), 1), 0)


In [ ]:
import json

with open('./datasets/irfane.geojson') as f:
    geojson_irfane = json.load(f)

#geojson_irfane

In [ ]:
ids = []
for idx, i in enumerate(geojson_irfane['features']):
    ids.append(i['properties']['id'])

In [ ]:
from sklearn import preprocessing
scaler = preprocessing.MinMaxScaler()

all_data_cholorpleth = []
for t in np.arange(TIME):
    data_cholorpleth = pd.DataFrame.from_records(data_polygons[t], columns=["longitude", "latitude", "cumul_score"])
    data_cholorpleth['id'] = ids
    
    #Normalize scores
    cs = data_cholorpleth[['cumul_score']].values #returns a numpy array
    cumul_score_scaled = scaler.fit_transform(cs) 
    data_cholorpleth["cumul_score_scaled"] = pd.DataFrame(cumul_score_scaled)
    
    all_data_cholorpleth.append(data_cholorpleth)

In [ ]:
#chose timestamp (between 0 and TIME)
t = 10
assert t < TIME and t >= 0

# map   
map_irfane_choropleth = folium.Map([med_lon_roi, med_lat_roi],  zoom_start=14)

# choropleth
folium.Choropleth(
    geo_data=geojson_irfane,
    name='choropleth',
    data=all_data_cholorpleth[t],
    columns=['id', 'cumul_score_scaled'],
    # see folium.Choropleth? for details on key_on
    key_on='feature.properties.id',
    fill_color='PuRd', # 'OrRd', 'PuBu', 'PuBuGn', 'PuRd', 'RdPu', 'YlGn', 'YlGnBu', 'YlOrBr', and 'YlOrRd'.
    fill_opacity=0.8,
    line_opacity=0.2,
    legend_name='score',
    highlight=True
).add_to(map_irfane_choropleth)

# layer control to turn choropleth on or off
folium.LayerControl().add_to(map_irfane_choropleth)

# display map
map_irfane_choropleth

#save as html
#map_irfane_choropleth.save("map" + str(t) + ".html")

## View geojson Files

In [ ]:
#geojson_path = './datasets/communes.geojson'
#geojson_path = './datasets/regions.geojson'
geojson_path = './datasets/maroc.geojson'


m = folium.Map(
    location=[31.0, -6.0],
    #tiles='Mapbox Bright',
    zoom_start=5  # Limited levels of zoom for free Mapbox tiles.
)

folium.GeoJson(
    geojson_path,
    name='geojson'
).add_to(m)


folium.LayerControl().add_to(m)


m

## generate Tiles with Skmob

In [ ]:
from skmob.tessellation import tilers
from skmob.utils import plot

rabattessellation = tilers.tiler.get("squared", base_shape="Rabat, Morocco", meters=150)

tess_style = {'color':'gray', 'fillColor':'gray', 'opacity':0.9}

map_f = plot.plot_gdf(rabattessellation, zoom=14, popup_features=['id'], style_func_args=tess_style, tiles='Open Street Map')

map_f

In [ ]:
from skmob.tessellation import tilers
import osmnx as ox

G = ox.graph_from_place('Beijing, China', which_result=2, network_type='drive')
ox.plot_graph(G)

nodes, _ = ox.graph_to_gdfs(G)
nodes.head()

tessellation = tilers.tiler.get("squared", base_shape=nodes, meters=500)
print(tessellation)